# Models and Maps

## Models

Let's again consider the car dataset from second notebook.

In that notebook we plotted *qsec* as a function of *hp*. However we might be interested a better model. Let's load the data.

In [211]:
library(tidyverse)

data(mtcars)

mtcars_tbl <- as_tibble(rownames_to_column(mtcars,var='model'))

str(mtcars_tbl)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	32 obs. of  12 variables:
 $ model: chr  "Mazda RX4" "Mazda RX4 Wag" "Datsun 710" "Hornet 4 Drive" ...
 $ mpg  : num  21 21 22.8 21.4 18.7 18.1 14.3 24.4 22.8 19.2 ...
 $ cyl  : num  6 6 4 6 8 6 8 4 4 6 ...
 $ disp : num  160 160 108 258 360 ...
 $ hp   : num  110 110 93 110 175 105 245 62 95 123 ...
 $ drat : num  3.9 3.9 3.85 3.08 3.15 2.76 3.21 3.69 3.92 3.92 ...
 $ wt   : num  2.62 2.88 2.32 3.21 3.44 ...
 $ qsec : num  16.5 17 18.6 19.4 17 ...
 $ vs   : num  0 0 1 1 0 1 0 1 1 1 ...
 $ am   : num  1 1 1 0 0 0 0 0 0 0 ...
 $ gear : num  4 4 4 3 3 3 3 4 4 4 ...
 $ carb : num  4 4 1 1 2 1 4 2 2 4 ...


Now let's fit three different linear models with `lm` from `stats`-package [[lm]](https://www.rdocumentation.org/packages/stats/versions/3.4.3/topics/lm).

First model will be `qsec ~ wt`, while second will be `qsec ~ hp`. Let's combine both of these effects into a third model `qsec ~ hp / wt`.

`summary` will show a summary of the model.

In [212]:
lm1_model <- function(data) lm(qsec ~ wt,      data=data)
lm2_model <- function(data) lm(qsec ~ hp,      data=data)
lm3_model <- function(data) lm(qsec ~ hp / wt, data=data)

summary(lm1_model(mtcars_tbl))
summary(lm2_model(mtcars_tbl))
summary(lm3_model(mtcars_tbl))


Call:
lm(formula = qsec ~ wt, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.3638 -1.0766  0.2051  0.8655  5.0298 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  18.8753     1.1025  17.120   <2e-16 ***
wt           -0.3191     0.3283  -0.972    0.339    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.789 on 30 degrees of freedom
Multiple R-squared:  0.03053,	Adjusted R-squared:  -0.00179 
F-statistic: 0.9446 on 1 and 30 DF,  p-value: 0.3389



Call:
lm(formula = qsec ~ hp, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.1766 -0.6975  0.0348  0.6520  4.0972 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 20.556354   0.542424  37.897  < 2e-16 ***
hp          -0.018458   0.003359  -5.495 5.77e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.282 on 30 degrees of freedom
Multiple R-squared:  0.5016,	Adjusted R-squared:  0.485 
F-statistic: 30.19 on 1 and 30 DF,  p-value: 5.766e-06



Call:
lm(formula = qsec ~ hp/wt, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.8659 -0.4913 -0.0812  0.3969  3.9757 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 21.387927   0.546219  39.156  < 2e-16 ***
hp          -0.041773   0.008020  -5.208 1.42e-05 ***
hp:wt        0.005028   0.001608   3.127    0.004 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.128 on 29 degrees of freedom
Multiple R-squared:  0.6272,	Adjusted R-squared:  0.6015 
F-statistic:  24.4 on 2 and 29 DF,  p-value: 6.103e-07


One can add arbitrary amount of terms into these models. There's plenty of other models in R libraries one might want to use.

# Nesting

Let's say we want to calculate the same models for each group specified by a cylinder. 

This means we need to do iteration over the groups and for this to work, we should split the data into chunks that will be iterated over. 

To do this we can use the `nest`-function ([[nest]](http://tidyr.tidyverse.org/reference/nest.html)).

In [213]:
mtcars_nested <- mtcars_tbl %>%
    # Convert cyl into a factor
    mutate_at(vars(cyl),as.factor) %>%
    # Group by cyl
    group_by(cyl) %>%
    # Nest the data
    nest()

print(mtcars_nested)

# A tibble: 3 x 2
  cyl   data              
  <fct> <list>            
1 6     <tibble [7 × 11]> 
2 4     <tibble [11 × 11]>
3 8     <tibble [14 × 11]>


This produces a `tibble` where all data is stored in a column of a type `list` and name *data*.

## Maps

Now that we have our list to iterate over, we can use `map` to do the iteration.

`map` is provided by the purrr-package. There are variants of it based on the return value of the used function. 

In this case we receive the results for a model (that are a list), so we want to use the `map`-function that creates a list from the outputs [[map]](http://purrr.tidyverse.org/reference/map.html).

Let's only focus on model no. 3, as that is the most interesting. 

In [214]:
# Map each data to model, pipe resulting fits to summary-function
map(mtcars_nested$data,lm3_model) %>%
    map(summary)

[[1]]

Call:
lm(formula = qsec ~ hp/wt, data = data)

Residuals:
      1       2       3       4       5       6       7 
-0.2463 -0.5001  0.8347  0.7807 -0.8807 -0.2807  0.2923 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 20.531267   1.855883  11.063  0.00038 ***
hp          -0.110793   0.023113  -4.793  0.00869 ** 
hp:wt        0.029015   0.008187   3.544  0.02392 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7995 on 4 degrees of freedom
Multiple R-squared:  0.8537,	Adjusted R-squared:  0.7806 
F-statistic: 11.67 on 2 and 4 DF,  p-value: 0.0214


[[2]]

Call:
lm(formula = qsec ~ hp/wt, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.90046 -0.63004  0.06546  0.66804  1.98746 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 20.686326   1.626110  12.721 1.37e-06 ***
hp          -0.076533   0.028019  -2.732   0.0258 *  
hp:wt        0.025052

A more *tidyverse*-approach to using the `map` is to use it with `mutate` to store the fits into a new columns. This makes it easy to run multiple models and store their results.

In [215]:
mtcars_nested <- mtcars_nested %>%
    mutate(
        model3=map(.$data, lm3_model)
    )

# Check structure
print(mtcars_nested)

# A tibble: 3 x 3
  cyl   data               model3  
  <fct> <list>             <list>  
1 6     <tibble [7 × 11]>  <S3: lm>
2 4     <tibble [11 × 11]> <S3: lm>
3 8     <tibble [14 × 11]> <S3: lm>


Package `broom` comes with nice functions `tidy` and `glance` that can be used to obtain coefficients or tests of the models in nice tibbles [[broom vignette]](https://cran.r-project.org/web/packages/broom/vignettes/broom.html).

In [216]:
library(broom)

tidy(mtcars_nested$model3[[1]])

glance(mtcars_nested$model3[[1]])

term,estimate,std.error,statistic,p.value
(Intercept),20.53126670,1.855882940,11.062803,0.0003796612
hp,-0.11079297,0.023113340,-4.793464,0.0086895335
hp:wt,0.02901531,0.008186876,3.544125,0.0239245317


r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
0.8537276,0.7805915,0.799514,11.67312,0.0213956,3,-6.407656,20.81531,20.59895,2.556891,4


Let's use `tidy` to get the model parameters.

In [217]:
mtcars_nested <- mtcars_nested %>%
    mutate(
        model1_coefs=map(model3,tidy),
        model2_coefs=map(model3,tidy),
        model3_coefs=map(model3,tidy)
    )

print(mtcars_nested)

# A tibble: 3 x 6
  cyl   data               model3   model1_coefs   model2_coefs   model3_coefs 
  <fct> <list>             <list>   <list>         <list>         <list>       
1 6     <tibble [7 × 11]>  <S3: lm> <data.frame [… <data.frame [… <data.frame …
2 4     <tibble [11 × 11]> <S3: lm> <data.frame [… <data.frame [… <data.frame …
3 8     <tibble [14 × 11]> <S3: lm> <data.frame [… <data.frame [… <data.frame …


Let's limit ourselves to coefficients and use `unnest` to unnest the coefficients.

In [218]:
mtcars_model3 <- mtcars_nested %>%
    select(cyl,model3_coefs) %>%
    unnest(model3_coefs)

print(mtcars_model3)

# A tibble: 9 x 6
  cyl   term         estimate std.error statistic  p.value
  <fct> <chr>           <dbl>     <dbl>     <dbl>    <dbl>
1 6     (Intercept)  20.5      1.86         11.1  3.80e- 4
2 6     hp           -0.111    0.0231       -4.79 8.69e- 3
3 6     hp:wt         0.0290   0.00819       3.54 2.39e- 2
4 4     (Intercept)  20.7      1.63         12.7  1.37e- 6
5 4     hp           -0.0765   0.0280       -2.73 2.58e- 2
6 4     hp:wt         0.0251   0.00827       3.03 1.63e- 2
7 8     (Intercept)  20.2      0.555        36.3  8.24e-13
8 8     hp           -0.0320   0.00385      -8.32 4.49e- 6
9 8     hp:wt         0.00393  0.000788      4.98 4.13e- 4
